# Evaluation rerun

the evaluation failed for some for questions, but we have the model answers so I want to rerun the evaluation

In [ ]:
# | default_exp rerun_eval

In [ ]:
# | export
from cat_tools.eval import (
    similarity_scorer,
    conf_variants,
    prepare_declarative_memory,
    CatModel,
)
import weave
import pandas as pd
from tqdm.auto import tqdm
import polars as pl
from polars import col as c
from cat_tools.client import SuperCatClient
from datetime import datetime

In [ ]:
# | export
weave_client = weave.init("smart-drying-unitus/declarative-eval")

weave: weave version 0.52.6 is available!  To upgrade, please run:
weave:  $ pip install weave --upgrade
weave: Logged in as Weights & Biases user: smassaro-unitus.
weave: View Weave data at https://wandb.ai/smart-drying-unitus/declarative-eval/weave


In [ ]:
# | export
calls_query = {
    "$expr": {
        "$and": [
            {"$gt": [{"$getField": "started_at"}, {"$literal": 1756801780.338}]},
            {
                "$or": [
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "019927e9-0d45-7a20-a70c-ad6886ffdb46"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "019926be-3e2c-7041-9927-26b76aa3fdb9"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "019925fa-39ee-7f57-b923-eb9cdb36c1e6"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "0199175a-03ae-7809-8292-f22c2320f1ee"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "01990fbd-ae07-74cc-9b8e-9e5ae323d958"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "019910d8-995a-78ed-8151-afbd5a5e4223"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "0199115d-6ac3-7422-815a-c2b56b7decae"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "01991448-bdfd-762d-ab6c-c4851521d521"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "01991061-7220-7a2a-aa03-a14bd6c8a003"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "01992df7-3180-7fec-8e3c-407c90711749"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "01992e00-da71-79a6-825e-777d99248eab"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "01992e10-d000-7bca-a342-d5b853621cf7"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "01992f3c-f2ed-7ed9-8881-2c99a8747d57"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "01992bc3-5ca2-7c50-b467-788ff19ab9b1"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "01993382-e1d3-75cc-9374-c2d111531b76"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "019934d5-e6d1-7ef5-8091-fee12873b766"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "019934da-fd3e-79c7-a546-ee1b92b4e270"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "019934dc-b009-7ce1-8b4a-b65d6932e994"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "0199353e-6253-7c91-a6e3-003c4f04779a"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "0199353f-3d33-79df-aa87-e3069d9ce80c"},
                        ]
                    },
                    {
                        "$eq": [
                            {"$getField": "id"},
                            {"$literal": "0199357c-2636-7c3a-9bd8-a010effbd164"},
                        ]
                    },
                ]
            },
        ]
    }
}

In [ ]:
# | export
def parse_calls(calls):
    outs = []
    for call in tqdm(list(calls)):
        model_name = call.inputs["model"].name
        has_mem = "NO memory" not in call.display_name
        for row in call.children():
            out = {}
            if inp := row.inputs.unwrap():
                if inp := inp.get("example", False):
                    out["question_id"] = inp["id"]
                    out["question"] = inp["prompt"]
                    out["expected_answer"] = inp["target"]
            if output := row.output:
                output = output.unwrap()
                if "output" in output:
                    if "text_clean" in output["output"]:
                        out["model_answer"] = output["output"]["text_clean"]
                    if "scores" in output:
                        if "CatEmbeddingSimilarityScorer" in output["scores"]:
                            out["similarity_score"] = output["scores"][
                                "CatEmbeddingSimilarityScorer"
                            ]["similarity_score"]

            out["model_name"] = model_name
            out["weave_call"] = row
            out["has_declarative_memory"] = has_mem
            outs.append(out)
    return pd.DataFrame(outs)

In [ ]:
calls = weave_client.get_calls(
    filter={"trace_roots_only": True},
    query=calls_query,
    sort_by=[{"field": "started_at", "direction": "desc"}],
)

In [ ]:
df = parse_calls(calls)

  0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
df

,question_id,question,expected_answer,model_answer,similarity_score,model_name,weave_call,has_declarative_memory
0,239.0,Che differenza c'è tra un prodotto disidratato...,La differenza fondamentale tra un prodotto dis...,Differenze principali:\n\n- Contenuto d’acqua/...,0.931725,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True
1,221.0,Come si valuta oggettivamente la capacità di r...,La capacità di reidratazione di un prodotto si...,Oggettivamente si valuta con il Rapporto di Re...,0.946931,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True
2,73.0,Quale impatto ha l'essiccazione convettiva sul...,L'impatto dell'essiccazione convettiva sul val...,L’essiccazione convettiva (aria calda) può rid...,0.921666,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True
3,77.0,Come è possibile controllare e ridurre la form...,Per ridurre la formazione della crosta è fonda...,Per limitare la “crosta” (case hardening) occo...,0.889175,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True
4,20.0,Per quale motivo prodotti come erbe e spezie v...,"Prodotti come erbe e spezie, il cui valore eco...",Per preservare la qualità aromatica. Erbe e sp...,0.899095,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True
...,...,...,...,...,...,...,...,...
3643,332.0,Come vengono caricati e scaricati i contenitor...,I contenitori del prodotto vengono caricati e ...,I contenitori vengono caricati e scaricati att...,0.983811,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True
3644,333.0,In che modo questo essiccatore gestisce il ric...,Questo essiccatore gestisce il ricircolo dell'...,Gestisce il ricircolo tramite un passaggio ret...,0.924412,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True
3645,334.0,Perché le pareti dell'essiccatore sono realizz...,Le pareti dell'essiccatore sono realizzate con...,"Per garantire isolamento, efficienza e durata....",0.892559,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True
3646,335.0,È possibile impostare diverse temperature per ...,"Sì, il pannello di controllo di questo essicca...",Sì. Il pannello di controllo consente di impos...,0.956343,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,True


In [ ]:
call = calls[0]

In [ ]:
call.display_name

'09-03 15:20 Eval - deepseek_best NO memory'

In [ ]:
df_pl = pl.from_pandas(df.drop(columns=["weave_call"]))

In [ ]:
df_pl.group_by(c.model_name, c.has_declarative_memory).agg(
    frac_answers=c.model_answer.is_not_null().sum() / 336,
    frac_scores=c.similarity_score.is_not_null().sum() / 336,
).sort(c.has_declarative_memory, c.model_name)

model_name,has_declarative_memory,frac_answers,frac_scores
str,bool,f64,f64
"""deepseek_best""",false,0.619048,0.619048
"""gemini_best""",false,1.0,1.0
"""gemma_best""",false,1.0,1.0
"""openai_best""",false,1.0,1.0
"""qwen_best""",false,1.0,1.0
"""deepseek_best""",true,1.0,1.0
"""gemini_best""",true,1.0,1.0
"""gemma_best""",true,1.0,1.0
"""openai_best""",true,1.0,1.0


In [ ]:
df_pl.group_by(c.model_name, c.has_declarative_memory, c.question_id).agg(
    pl.len()
).filter(c.len > 1)

model_name,has_declarative_memory,question_id,len
str,bool,f64,u32
"""deepseek_best""",false,141.0,2
"""deepseek_best""",false,161.0,2
"""qwen_best""",false,174.0,2
"""qwen_best""",true,324.0,2
"""deepseek_best""",false,64.0,2
…,…,…,…
"""deepseek_best""",true,127.0,2
"""deepseek_best""",true,64.0,2
"""deepseek_best""",true,72.0,2


In [ ]:
df_pl.filter(c.question_id.is_not_null(), c.model_answer.is_not_null()).group_by(
    c.model_name, c.has_declarative_memory
).agg(pl.len()).with_columns(n_missing=336 - c.len)

model_name,has_declarative_memory,len,n_missing
str,bool,u32,u32
"""openai_best""",false,336,0
"""gemini_best""",true,336,0
"""qwen_best""",true,336,0
"""gemini_best""",false,336,0
"""gemma_best""",true,336,0
"""deepseek_best""",false,208,128
"""openai_best""",true,336,0
"""deepseek_best""",true,336,0
"""qwen_best""",false,336,0


In [ ]:
df_pl.group_by(c.model_name, c.has_declarative_memory, c.question_id).agg(
    all_missing=c.model_answer.is_null().all(), len=pl.len()
).sort(c.all_missing, descending=True)

model_name,has_declarative_memory,question_id,all_missing,len
str,bool,f64,bool,u32
"""gemma_best""",true,null,true,2
"""gemini_best""",false,null,true,2
"""qwen_best""",true,null,true,2
"""qwen_best""",false,null,true,3
"""gemini_best""",true,null,true,2
…,…,…,…,…
"""openai_best""",false,263.0,false,1
"""gemma_best""",true,209.0,false,1
"""openai_best""",false,212.0,false,1


In [ ]:
df_pl.group_by(c.model_name, c.has_declarative_memory, c.question_id).agg(
    all_missing=c.model_answer.is_null().all(), len=pl.len()
).sort(c.all_missing, descending=True)

model_name,has_declarative_memory,question_id,all_missing,len
str,bool,f64,bool,u32
"""deepseek_best""",false,null,true,1
"""qwen_best""",true,null,true,2
"""gemma_best""",true,null,true,2
"""deepseek_best""",true,null,true,2
"""gemini_best""",true,null,true,2
…,…,…,…,…
"""openai_best""",true,306.0,false,1
"""gemma_best""",true,40.0,false,1
"""gemma_best""",true,216.0,false,1


In [ ]:
df_pl.filter(c.question_id.is_not_null()).with_columns(
    all_missing=c.model_answer.over(
        [c.question_id, c.model_name, c.has_declarative_memory]
    )
    .is_null()
    .sum(),
    n_questions=pl.len().over([c.question_id, c.model_name, c.has_declarative_memory]),
).filter(c.n_questions > 1).sort(c.question_id, c.model_name, c.has_declarative_memory)

question_id,question,expected_answer,model_answer,similarity_score,model_name,has_declarative_memory,all_missing,n_questions
f64,str,str,str,f64,str,bool,u32,u32
0.0,"""Che cos'è la disidratazione a …","""La disidratazione a flusso di …","""La disidratazione a flusso di …",0.866623,"""gemma_best""",false,397,2
0.0,"""Che cos'è la disidratazione a …","""La disidratazione a flusso di …",null,null,"""gemma_best""",false,397,2
0.0,"""Che cos'è la disidratazione a …","""La disidratazione a flusso di …",""" La disidratazione a flusso d…",0.880706,"""qwen_best""",false,397,2
0.0,"""Che cos'è la disidratazione a …","""La disidratazione a flusso di …",null,null,"""qwen_best""",false,397,2
1.0,"""Qual è l'obiettivo principale …","""L'obiettivo principale del pro…","""L'obiettivo principale del pro…",0.836221,"""gemma_best""",false,397,2
…,…,…,…,…,…,…,…,…
333.0,"""In che modo questo essiccatore…","""Questo essiccatore gestisce il…",null,null,"""gemini_best""",false,397,2
334.0,"""Perché le pareti dell'essiccat…","""Le pareti dell'essiccatore son…","""Certamente. La scelta di utili…",0.907079,"""gemini_best""",false,397,2
334.0,"""Perché le pareti dell'essiccat…","""Le pareti dell'essiccatore son…",null,null,"""gemini_best""",false,397,2


In [ ]:
missing = (
    df_pl.filter(c.question_id.is_not_null())
    .group_by(
        c.model_name,
        c.has_declarative_memory,
        c.question,
        c.expected_answer,
        c.question_id,
    )
    .agg(
        all_missing=c.model_answer.is_null().all(),
    )
    .filter(c.all_missing)
    .group_by(c.model_name, c.has_declarative_memory)
    .agg(
        question=c.question,
        target=c.expected_answer,
        id=c.question_id.cast(pl.Int64),
        n=pl.len(),
    )
    .sort(c.has_declarative_memory)
)

In [ ]:
missing

model_name,has_declarative_memory,question,target,id,n
str,bool,list[str],list[str],list[i64],u32


In [ ]:
# | export
def get_dataset(missing):
    datasets = []
    for row in missing.iter_rows(named=True):
        dataset = []
        for id, q, t in zip(row["id"], row["question"], row["target"]):
            if id is None:
                continue
            dataset.append({"id": id, "prompt": q, "input": q, "target": t})
        datasets.append(dataset)
    return datasets

In [ ]:
missing = missing.with_columns(dataset=pl.Series(get_dataset(missing), dtype=pl.Object))

In [ ]:
# | export
def get_missing_df(weave_client, query=calls_query):
    calls = list(
        weave_client.get_calls(
            filter={"trace_roots_only": True},
            query=query,
            sort_by=[{"field": "started_at", "direction": "desc"}],
        )
    )
    df = parse_calls(calls)
    df_pl = pl.from_pandas(df.drop(columns=["weave_call"]))
    missing = (
        df_pl.filter(c.question_id.is_not_null())
        .group_by(
            c.model_name,
            c.has_declarative_memory,
            c.question,
            c.expected_answer,
            c.question_id,
        )
        .agg(
            all_missing=c.model_answer.is_null().all(),
        )
        .filter(c.all_missing)
        .group_by(c.model_name, c.has_declarative_memory)
        .agg(
            question=c.question,
            target=c.expected_answer,
            id=c.question_id.cast(pl.Int64),
            n=pl.len(),
        )
        .sort(c.has_declarative_memory)
    )
    missing = missing.with_columns(
        dataset=pl.Series(get_dataset(missing), dtype=pl.Object)
    )
    return missing

In [ ]:
# | export
async def run_missing(missing):
    client = SuperCatClient()
    time = datetime.now().strftime("%m-%d %H:%M")
    eval_name = f"{time} Eval"
    cat_has_memory = False
    for row in missing.iter_rows(named=True):
        dataset = row["dataset"]
        model_name = row["model_name"]
        has_mem = row["has_declarative_memory"]
        if cat_has_memory != has_mem:
            if has_mem:
                print("Preparing declarative memory...")
                prepare_declarative_memory(client)
            else:
                client.wipe_declarative_memory()
            cat_has_memory = has_mem
        print(f"Running {model_name} with memory={has_mem} on {len(dataset)} examples")
        model = CatModel(
            model_name, conf_variants[model_name], has_declarative_memory=has_mem
        )
        evaluation = weave.Evaluation(
            dataset=dataset,
            scorers=[similarity_scorer],
            name=eval_name,
        )
        await evaluation.evaluate(
            model,
            __weave={
                "display_name": f"{eval_name} - {model_name} {'memory' if has_mem else 'NO memory'}"
            },
        )

In [ ]:
await run_missing(missing.head(1))

Running gemma_best with memory=False on 336 examples
